In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1" 
from misic.misic import *
from misic.extras import *
from skimage.io import imsave,imread
from skimage.transform import resize,rescale

import time, os, sys
import skimage.io
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300
import edt

In [ ]:
from omnipose.core import diameters, dist_to_diam

### Load test dataset

In [ ]:
from pathlib import Path
from cellpose import io
basedir = '/home/kcutler/DataDrive/omnipose_all/Fig1_comparison/misic'
io.check_dir(basedir)
basedir = '/home/kcutler/DataDrive/omnipose_test'
mask_filter = '_masks'
img_names = io.get_image_files(basedir,mask_filter,look_one_level_down=True)
mask_names,flow_names = io.get_label_files(img_names, mask_filter)
#annoyingly, the names are not sorted properly because the paths are all different for the test folder...
def getname(path,suffix='_masks'):
    return os.path.splitext(Path(path).name)[0].replace(suffix,'')
mask_names = sorted(mask_names,key=getname)
img_names = sorted(img_names,key=getname)
imgs = [skimage.io.imread(f) for f in img_names]
masks_gt = [skimage.io.imread(f) for f in mask_names]

In [ ]:
# check length of test set 
len(masks_gt) 

### Run MiSiC
I had to make a few tweaks to MiSiC to get it to install. Some optimizations for my images were made following advice from the author of MiSiC. 

In [ ]:
from skimage.transform import rescale,resize
from skimage.util import random_noise
from skimage.filters import gaussian
from cellpose import transforms

import cv2
Y = []

do_size = 0
invert = 0

if do_size:
    diams = [diameters(m) for m in masks_gt]

# Initialize MiSiC
# my branch of MiSiC allows for the loading of custom models 
mseg = MiSiC(model_dir='/home/kcutler/DataDrive/misic_etc/new_misic_model_kevin_ep100_bs8.h5')
# mseg = MiSiC(model_dir='/home/kcutler/DataDrive/misic_etc/new_misic_model_kevin.h5')
# mseg = MiSiC()
for idx in range(len(imgs)):
    
    im = imgs[idx].copy()
    sr,sc = im.shape
    
    # added per Swapnesh's recommendations, helps a lot, holy cow! 
    # blur not so much, bit a tiny bit of noise is good
    if 1:
        # im = gaussian(im,1.5)
        im = transforms.normalize99(im,omni=True)
        im = random_noise(im,mode ='gaussian',var = 0.000001)
        # im = add_noise(im, sensitivity = .1, invert = invert)

    if do_size: # my custom model not trained with resizing in mind, variance in width is small 
        #input the approximate mean width of microbe under consideration
        mean_width = diams[idx]/2
        # compute scaling factor
        scale = (10/mean_width)
        # preprocess using inbuit function or if you are feeling lucky use your own preprocessing
        im = rescale(im,scale,preserve_range = True)
    
    
    # Parameters that need to be changed
    ## Ideally, use a single image to fine tune two parameters : mean_width and noise_variance (optional)
    # add local noise
    # img = add_noise(im, sensitivity = .5, invert = True)
    img = im.copy()

    # segment
    yp = mseg.segment(img,invert = invert)
    # yp = mseg.segment_auto(im) broken
    if do_size:
        yp = resize(yp,(sr,sc))
        img = resize(img,(sr,sc))
    # masks  = postprocess_ws(cv2.resize(img,[yp.shape[1],yp.shape[0]]),yp)
    masks  = postprocess_ws(cv2.resize(img,(yp.shape[1],yp.shape[0])),yp[...,0])
    
    Y.append(masks)
    plt.imshow(np.hstack((img,yp[:,:,0],yp[:,:,1])))
    plt.axis('off')
    plt.show()
    plt.imshow(masks)
    plt.axis('off')
    plt.show()
    savedir = '/home/kcutler/DataDrive/omnipose_all/Fig1_comparison/misic_trained/'
    io.check_dir(savedir)
    io.imsave(os.path.join(savedir,getname(mask_names[idx])+'_masks.tif'),masks)